In [17]:
import pickle
import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from scipy.io.arff import loadarff 
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [18]:
from sklearn.pipeline import make_pipeline

In [19]:
import mlflow

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("credit-class-prediction")

<Experiment: artifact_location='s3://credit-card-mlops-orchestration/2', creation_time=1690314745036, experiment_id='2', last_update_time=1690314745036, lifecycle_stage='active', name='credit-class-prediction', tags={}>

In [20]:
def read_dataframe(filename: str):
    data = loadarff('./data/credit_data.arff')
    df = pd.DataFrame(data[0])
    le = LabelEncoder()
    df['class'] = le.fit_transform(df['class'])
    return df


def prepare_dictionaries(df: pd.DataFrame):
    numerical = ['duration', 'credit_amount', 'installment_commitment', 'residence_since', 'age', 'existing_credits', 'num_dependents']
    dicts = df[numerical].to_dict(orient='records')
    return dicts

In [25]:
df = read_dataframe('./data/credit_data.arff')
df_train ,df_val = train_test_split(df, test_size=0.2, random_state=42)  

target = 'class'
y_train = df_train[target].values
y_val = df_val[target].values

dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_val)

In [26]:
with mlflow.start_run():
    params = dict(random_state=0)
    mlflow.log_params(params)

    pipeline = make_pipeline(
        DictVectorizer(),
        LogisticRegression(**params, n_jobs=-1)
    )

    pipeline.fit(dict_train, y_train)
    y_pred = pipeline.predict(dict_val)

    rmse = mean_squared_error(y_pred, y_val, squared=False)
    print(params, rmse)
    mlflow.log_metric('rmse', rmse)

    mlflow.sklearn.log_model(pipeline, artifact_path="model")

{'random_state': 0} 0.5099019513592785


/Users/kylekawczynski/.local/share/virtualenvs/web-service-mlflow-03PBvZVj/lib/python3.9/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/Users/kylekawczynski/.local/share/virtualenvs/web-service-mlflow-03PBvZVj/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
